# Algorithms for determinig cluster order picking strategies

This notebook explains and demonstrates the algorithms for finding strategies for "Cluster Order Picking" proposed by Goetschalckx and Ratliff (1988)

## Imports

In [1]:
from math import sqrt

## Beschreibung des Problems

### Ziel
Minimierung der Komissionierdauer für die gesamte Bestellung (picking-time)

### Parameter
- a) round-trip-walking-time
- b) time-to-start-stop ($f$)
- c) driving time (independent from stops)
- d) time to pick items (independent from stops)
- e) time to stack items (independent from stops)
- distance from beginning of ailse to pick $i$: $a_{i}$
- distance from beginning of aisle to stop $j$: $z_{j}$
- width of aisle $A$
- distance between stop $j$ and pick $i$: $d_{ji }$ (Siehe Distanzmaße)
- number of items to be picked in aisle $N$
- Laufgeschwindigkeit des Kommissionierers zwischen Regalplatz und Komissionierfahrzeug: $v$
- total time per stop (roundtriptime to picks) $c_{j}=f+ 2 \sum_{i \in P_{j}} d_{ij}/v$

### Distanzmaße
Der Algorithmus kann mit zwei verschiedenen Distanzmaßen verwendet werden. Praktisch sind die Ergebnisse, die mit den beiden Distanzmaßen meist identisch oder weichen nur sehr gering voneinander ab. Für den Praktischen Einsatz wird deshalb *Rectilinear*, aufgrund der geringeren Laufzeit empfohlen. (Zitat)

#### Euclidean
Distanz von stop $j$ zu pick $i$ bei einer Gangbreite von $A$ nach *euklidischem* Abstand:

$$ d_{ij} = \sqrt{(a_{i}-z_{j})^2+ (A/2)^2} $$

In [2]:
def distance_euklidean(stop_position, pick_position, aisle_width):
    return sqrt((pick_position - stop_position)**2+(aisle_width/2)**2)

In [3]:
# Example:
distance_euklidean(3, 4, 7)

3.640054944640259

#### Rectilinear
Distanz von stop $j$ zu pick $i$ bei einer Gangbreite von $A$ nach *rectilinearem* Abstand:

$$ d_{ij} = |(a_{i}-z_{j})| + (A/2) $$

In [4]:
def distance_rectilinear(stop_position, pick_position, aisle_width):
    return abs(pick_position - stop_position) + aisle_width/2

In [5]:
# Example:
distance_rectilinear(3, 7, 4)

6.0

## Kostenfunktion
Da Ziel ist die Komissionierzeit zu minimieren, misst die Kostenkunktion die Dauer des Komissioniervorgangs.
Die Kosten $c_{j}$ für eine Menge zu pickender Items $P_{j}$ bei einem Stop $j$ sind gegeben durch:
$$ c_{j}=f+ 2 \sum_{i \in P_{j}} d_{ij}/v $$
wobei die Berechnung abhängig von der Wahl des Distanzmaßes ist.

In [6]:
def cost_measure(item_positions, stop_position, walking_speed, time_to_start_stop, aisle_width,
                 distance_measure):
    return time_to_start_stop + 2*sum([distance_measure(stop_position, i, aisle_width)/walking_speed for i in item_positions])

In [7]:
# Example
cost_measure([1,2,3,4], 2, 1, 2, 4, distance_euklidean)

20.60112615949154

## Optimale Stops
Algorithmus zur Bestimmung der Optimalen Stopstrategie bei frei positionierbaren stops
### Eigenschaften:
1. Items werden immer beim Stop mit der geringsten Entfernung aufgeladen.

2. Items picked at the same stop are in one line on both sides by the order which does not contain cases picked at other stops.
    - Blockstruktur: Alle Alle Items, die bei einem Stop aufgladen werden, liegen sind in einer ununterbrochenen Reihe (Block)

3. Optimale Halteposition $z_{j}$ bei einer vorgegebenen Menge von zu ladenden Items $P_{j}$ mit *euklidischem* Distanzmaß (Iterativer Prozess): ( the optimal stoplocation ($z_{j}$) for a given set of picks $(P_{j})$ under *euclidean* distannce (single facility *Euclidean* location problem) is determined by (iterative procedure):
$$z_{j}=\frac{ \sum_{i \in P_{j}} a_{i}/d_{ij} }{\sum_{i \in P_{j}} 1/d_{ij}}$$

In [8]:
def optimal_stop_euclidean(item_positions, aisle_width, starting_point=0, iterations=10):
    z = starting_point
    for _ in range(iterations):
        distances = [distance_euklidean(z, a, aisle_width) for a in item_positions]
        z = sum(a/d for (a,d) in zip(item_positions, distances)) / sum(1/d for d in distances)
    return z

In [9]:
# Example
optimal_stop_euclidean([2,5,7], 4)

4.834449438734148

4. Optimale Halteposition $z_{j}$ bei einer vorgegebenen Menge von zu ladenden Items $P_{j}$ (aufsteigend sortiert) mit *rectilinearem* Distanzmaß:
    - $|P_{j}|$ ungerade: Position des Items in der Mitte
    - $|P_{j}|$ ungerade: Alle Positionen zwischen den beiden Mittleren sind optimal (Implementierung wählt die Mitte)

In [10]:
def optimal_stop_rectilinear(item_positions, aisle_width):
    sorted = list(item_positions) # copy list
    length = len(sorted)
    sorted.sort()
    if length % 2:
        return sorted[int(length/2)]
    else:
        return (sorted[int(length/2)] + sorted[int(length/2) -1])/2

In [11]:
# Example
optimal_stop_rectilinear([2,5,7,100,1000], 4)

7

## Dynamische Programmierung
Implementierung, des Algorithmus

In [12]:
def optimal_stops(item_positions, aisle_width,
                  walking_speed,
                  time_to_start_stop,
                  distance_measure=distance_euklidean,
                  optimal_stop_func=optimal_stop_euclidean,
                  cost_measure=cost_measure):
    """
    Calculates optimal number of stops and items picked per stop
    :returns: list containing the optimal item sets and the target value for the calculated solution
    """
    # Adding artificial first elemenmt
    sorted_items = [0] + list(item_positions)
    sorted_items.sort()
    
    # Adding artificial first element
    path_lengths = [float('inf')] * len(sorted_items)
    path_lengths[0] = 0
    optimal_predecessors = [0] * len(sorted_items)
    
    for index, item in enumerate(sorted_items[1:], 1):
        #print('Item: {} at index: {}'.format(item, index))
        for pred_index, pred_item in enumerate(sorted_items[:index]):
            #print(pred_item, pred_index)
            item_set = sorted_items[pred_index+1:index+1]
            optimal_location = optimal_stop_func(item_set, aisle_width=aisle_width)
            arc_length = cost_measure(
                item_set,
                stop_position=optimal_location,
                walking_speed=walking_speed,
                time_to_start_stop=time_to_start_stop,
                aisle_width=aisle_width,
                distance_measure=distance_measure
            )
            #print('\tItem Set: {}, Optimal Location: {}, Arc Length: {}'.format(item_set, optimal_location, arc_length))
            #print('\tOld Path Length: {}, New Path length: {}'.format(path_lengths[index], (path_lengths[pred_index] + arc_length)))
            if path_lengths[index] > (path_lengths[pred_index] + arc_length):
                #print('\tnew: {}, pred: {}'.format(index, pred_index))
                path_lengths[index] = path_lengths[pred_index] + arc_length
                optimal_predecessors[index] = pred_index
    
    target_value = path_lengths[-1]
    item_sets = list()
    index = len(optimal_predecessors) - 1
    while index > 0:
        pred_index = optimal_predecessors[index]
        item_sets.append(tuple(sorted_items[pred_index+1:index+1]))
        index = pred_index
    item_sets.reverse()
    return item_sets, target_value

In [13]:
# Example
item_positions = [2000,2000,2, 1, 0]
optimal_stops(item_positions, aisle_width=2, walking_speed=10, time_to_start_stop=1)

([(0, 1, 2), (2000, 2000)], 3.1656854249520645)